In [18]:
import pandas as pd

In [19]:
df_train_raw = pd.read_csv("./train.csv",index_col="id")
# convert date into pandas datetime object
ds_train_datetime = pd.to_datetime(df_train_raw["date"], dayfirst=True, format="%d/%m/%Y %H:%M")

# construct a new dataframe to do analysis
df_train_data = df_train_raw.copy()
df_train_data["date"] = ds_train_datetime
ts_train_data = df_train_data.set_index("date")
ts_train_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
speed,14006.0,32.779118,13.573813,2.573417,19.301089,36.580595,45.877665,53.161286


In [20]:
import datetime as dt
import numpy as np
import sys
import time

In [21]:
from statsmodels.tsa.arima.model import ARIMA

In [22]:
ts_nan = pd.Series(np.nan, index=pd.date_range('2017', end='2019', freq='H'), name='nan')
df_base = pd.concat([ts_nan, ts_train_data], axis=1)
ds_base = df_base.drop(columns=["nan"])

In [23]:
ds_real_pred = ds_base.copy()

In [24]:
myDict = {}
myResid = {}
td_28day = dt.timedelta(days=28)
td_1day= dt.timedelta(days=1)
td_1hour = dt.timedelta(hours=1)

In [25]:
end_date = dt.datetime(2018, 1, 1)
n_step_to = dt.datetime(2018, 12, 31,23)
n_step = int((n_step_to - end_date).total_seconds()/(3600))

thisTime = time.time()
for i in range(n_step):
    dt_ihour = dt.timedelta(hours=i)
    dt_end = end_date + dt_ihour

    dt_start = dt_end - td_28day
    ds_window = ds_real_pred[dt_start:dt_end]
#     print(ds_window.tail())
#     print(dt_end+td_1hour)
#     ds_window_norm , tuple_state = normalization(ds_window)
    # ds_window_norm_dweek = ds_first_month_norm.diff(7*24).dropna()

    my_order = (3,0,1)
#     my_order = (3,1)
    model = ARIMA(ds_window, order=my_order)
    model_fit = model.fit()

    predictions = model_fit.forecast(1)
    prediction_val = predictions
#     myDict[dt_end] = prediction_val    # I don't know why but it seems that, shifting one data , result better
    myDict[dt_end + td_1hour] = prediction_val 
    
    # replace missing value and keep on predict!
    print(dt_end,round(i*100/n_step,4),"%\0", end='\r', flush=True)
    if np.isnan(ds_real_pred.loc[dt_end+td_1hour].speed) == True:
        ds_real_pred.loc[dt_end+td_1hour] = prediction_val
#         print(dt_end, " is replace ", prediction_val)
    myResid[dt_end + td_1hour] = ds_real_pred.loc[dt_end + td_1hour] - prediction_val

ds_prediction = pd.Series(myDict)
ds_resid = pd.Series(myResid)
print("Done!! Time used:", (time.time() - thisTime)/60, "second")

/home/jupyter-cenz/.local/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


LinAlgError: LU decomposition error.

In [ ]:
df = pd.DataFrame(ds_prediction)
df.to_csv('predictions.csv',index=True)

In [ ]:
df_predictions_raw = pd.read_csv("./predictions.csv")
df_predictions_raw.columns = ["date","speed"]
# convert date into pandas datetime object 2018-12-31 22:00:00
df_predictions_datetime = pd.to_datetime(df_predictions_raw["date"], dayfirst=True, format="%Y-%m-%d %H:%M")

# construct a new dataframe to do analysis
df_predictions_data = df_predictions_raw.copy()
df_predictions_data["date"] = df_predictions_datetime
df_predictions_data = df_predictions_data.set_index("date")
df_predictions_data.describe().transpose()

In [ ]:
df_test_raw = pd.read_csv("./test.csv", index_col="id")
df_test_raw["speed"] = np.nan
# df_test_raw.columns = ["date"]
# convert date into pandas datetime object 2018-12-31 22:00:00
df_test_datetime = pd.to_datetime(df_test_raw["date"], dayfirst=True, format="%d/%m/%Y %H:%M")

# construct a new dataframe to do analysis
df_test_data = df_test_raw.copy()
df_test_data["date"] = df_test_datetime
df_test_data = df_test_data.set_index("date")
df_test_data.describe().transpose()

In [ ]:
# df_test_data, df_predictions_data
df_submit = pd.merge(df_predictions_data, df_test_data, how='inner', on="speed", left_index=True, right_index=True)

In [ ]:
df_submit.to_csv('test_submit.csv',index=True)